# Re route

## dataset set up

older dataset

In [1]:
# set up of the same dataset of performance test
import pandas as pd

path = r"C:\Users\ElifYilmaz\OneDrive - Enpal B.V\Desktop\New folder\Project\reroute, more samples\merged_df_bigger_sample.csv"
df = pd.read_csv(path)
df.columns = df.columns.str.lower()


# optionally turn it into net sc only 
#df = df [ df ['net_sc'] != 0].copy()

#separating into old positives, new positives and all negatives nb: would be better to mark also the older/ newer negatives maybe later
new_positives = df[ (df['old'] == 0) & (df['grosscontractsigned']== 1)].reset_index(drop=True)
old_positives =  df[df['old']==1].reset_index(drop=True)
rest_negatives = df[ df[ 'grosscontractsigned'] == 0].reset_index(drop=True)

negative_train = rest_negatives.sample(frac=0.7, random_state=42)
negative_test = rest_negatives.drop(negative_train.index)

# these is 4184. to keep the original ratio i have to add 210 new positives
test_new_positives = new_positives.sample(n=244, random_state = 42)
train_new_positives = new_positives.drop(test_new_positives.index)

# create final train and test
train = pd.concat( [negative_train, train_new_positives, old_positives])
test = pd.concat ( [test_new_positives, negative_test])

# boo dont forget to drop old mark
train = train.drop('old', axis=1)
test = test.drop('old', axis=1)

# for some reason selfipadone is missing on a few columns, later look but for now fill with 0 
train['selfipa_done'] = train['selfipa_done'].fillna(0)
test['selfipa_done'] = test['selfipa_done'].fillna(0)

#split 
X_train = train.drop('grosscontractsigned', axis=1)
y_train = train['grosscontractsigned']
X_test = test.drop('grosscontractsigned', axis=1)
y_test = test['grosscontractsigned']

newer dataset

In [5]:
import pandas as pd
path = r"C:\Users\ElifYilmaz\OneDrive - Enpal B.V\Desktop\New folder\Project\processed_data\merged_df_only_after_march.csv"

df = pd.read_csv(path)
df.columns = df.columns.str.lower()


# optionally turn it into net sc only 
#df = df [ df ['net_sc'] != 0].copy()
# since double layer model, we keep it as is

from sklearn.model_selection import train_test_split

# Stratified split to maintain class distribution
X = df.drop('grosscontractsigned', axis=1)
y = df['grosscontractsigned']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,
    stratify=y,
    random_state=42
)

print(f"Train set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")
print(f"\nTrain class distribution:")
print(y_train.value_counts(normalize=True))
print(f"\nTest class distribution:")
print(y_test.value_counts(normalize=True))

Train set size: 14364
Test set size: 3591

Train class distribution:
grosscontractsigned
0.0    0.96714
1.0    0.03286
Name: proportion, dtype: float64

Test class distribution:
grosscontractsigned
0.0    0.96714
1.0    0.03286
Name: proportion, dtype: float64


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10699 entries, 1 to 17954
Data columns (total 41 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   gross_fu                        10699 non-null  int64  
 1   gross_sc                        10699 non-null  int64  
 2   net_fu                          10699 non-null  float64
 3   net_sc                          10699 non-null  float64
 4   time_first_sc_to_first_net_fu   10699 non-null  float64
 5   electricitybill                 10699 non-null  float64
 6   heatingbill                     10699 non-null  float64
 7   grosscontractsigned             10699 non-null  float64
 8   selfipa_done                    10699 non-null  int64  
 9   zipregion_missing               10699 non-null  int64  
 10  evaluationtime_missing          10699 non-null  int64  
 11  desiredinstallationend_missing  10699 non-null  int64  
 12  electricitybill_missing         10699

In [6]:
# setting up further 
y_sc = (df['net_sc'] >= 1).astype(int)


##  Base model and Fisrt NN - 2 Layer Model

Step 1: Setup + Labels + Feature Lists

In [9]:
import numpy as np
import pandas as pd

# --- Step 1: define targets and feature sets ---

# Layer 1 target: whether lead reached at least one SC
y_sc = (df['net_sc'] >= 1).astype(int)

# Layer 2 target: contract signed (already split into y_train / y_test separately)
y_sign = df['grosscontractsigned']

# Features for layer 1: drop SC/FU outcome columns (leakage for this stage)
drop_cols_layer1 = [
    'gross_fu', 'gross_sc', 'net_fu', 'net_sc',
    'time_first_sc_to_first_net_fu', 'selfipa_done'
]
X_layer1 = X.drop(columns=drop_cols_layer1)

# Features for layer 2: keep everything (since SC/FU info is valid here)
X_layer2 = X.copy()

print("Shape of full dataset:", X.shape)
print("Layer 1 features:", X_layer1.shape[1])
print("Layer 2 features:", X_layer2.shape[1])

# Just to sanity check alignment
print("y_sc positive rate:", y_sc.mean().round(3))
print("y_sign positive rate:", y_sign.mean().round(3))


Shape of full dataset: (17955, 40)
Layer 1 features: 34
Layer 2 features: 40
y_sc positive rate: 0.596
y_sign positive rate: 0.033


Step 2: Metric functions

In [11]:
import numpy as np

def recall_at_percent(y_true, y_scores, top_percent=0.05):
    """
    Recall@% = fraction of all positives captured in the top % of ranked leads.
    """
    n_top = int(len(y_true) * top_percent)
    if n_top < 1: 
        n_top = 1
    # rank by predicted score
    idx = np.argsort(y_scores)[::-1][:n_top]
    return y_true.iloc[idx].sum() / y_true.sum()

def lift_at_percent(y_true, y_scores, top_percent=0.05):
    """
    Lift@% = (positive rate in top % of ranked leads) / (overall positive rate).
    """
    n_top = int(len(y_true) * top_percent)
    if n_top < 1:
        n_top = 1
    idx = np.argsort(y_scores)[::-1][:n_top]
    top_rate = y_true.iloc[idx].mean()
    overall_rate = y_true.mean()
    return top_rate / overall_rate if overall_rate > 0 else np.nan


Step 3: cross-validation loop.

Here we’ll:

create pipelines for baseline (logistic regression) and NN (MLP),

run StratifiedKFold,

train both layer 1 and layer 2 models inside each fold,

compute P_final = P_sc * P_sign_given_sc,

collect recall and lift at 1%, 5%, 10%.

In [13]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import pandas as pd

# Models
baseline_clf = LogisticRegression(class_weight='balanced', max_iter=5000)
nn_clf = MLPClassifier(hidden_layer_sizes=(64, 32), activation='relu',
                       early_stopping=True, random_state=42, max_iter=200)

# Wrap NN in a pipeline with scaling
nn_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("nn", nn_clf)
])

# CV setup
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

results = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_layer2, y_sign)):
    # Split indexes
    X1_train, X1_val = X_layer1.iloc[train_idx], X_layer1.iloc[val_idx]
    y1_train, y1_val = y_sc.iloc[train_idx], y_sc.iloc[val_idx]
    
    X2_train, X2_val = X_layer2.iloc[train_idx], X_layer2.iloc[val_idx]
    y2_train, y2_val = y_sign.iloc[train_idx], y_sign.iloc[val_idx]
    
    for model_name, model in [("Baseline", baseline_clf), ("NN", nn_pipeline)]:
        # ---- Layer 1 ----
        model1 = model.fit(X1_train, y1_train)
        p_sc = model1.predict_proba(X1_val)[:,1]
        
        # ---- Layer 2 ----
        # train only on those with y_sc=1 in training set
        mask_sc_train = (y1_train == 1)
        model2 = model.fit(X2_train[mask_sc_train], y2_train[mask_sc_train])
        p_sign_given_sc = model2.predict_proba(X2_val)[:,1]
        
        # ---- Final probability ----
        p_final = p_sc * p_sign_given_sc
        
        # ---- Metrics ----
        for pct in [0.01, 0.05, 0.10]:
            rec = recall_at_percent(y2_val, p_final, top_percent=pct)
            lift = lift_at_percent(y2_val, p_final, top_percent=pct)
            results.append({
                "fold": fold+1,
                "model": model_name,
                "percent": pct,
                "recall": rec,
                "lift": lift
            })

cv_results = pd.DataFrame(results)
summary = cv_results.groupby(["model","percent"]).agg(
    recall_mean=("recall","mean"), recall_std=("recall","std"),
    lift_mean=("lift","mean"), lift_std=("lift","std")
).reset_index()

summary


,model,percent,recall_mean,recall_std,lift_mean,lift_std
0,Baseline,0.01,0.049153,0.009283,5.043051,0.952480
1,Baseline,0.05,0.138983,0.035249,2.788202,0.707137
2,Baseline,0.10,0.240678,0.028488,2.407450,0.284956
3,NN,0.01,0.091525,0.056022,9.390508,5.747859
4,NN,0.05,0.288136,0.145309,5.780419,2.915104
5,NN,0.10,0.415254,0.173780,4.153699,1.738289


what these numbers mean  Two-layer cascade  The NN results you see already include both layers.  For each fold we did:  Train NN on Layer 1 (BC→SC) → get  𝑃 ( SC ) P(SC).  Train NN on Layer 2 (SC→Contract) → get  𝑃 ( sign ∣ SC ) P(sign∣SC). 
 Multiply: P(final sign)=P(SC)×P(sign∣SC).

The metrics are calculated on this final combined probability.  

Step 4: final benchmark on the untouched test set.

In [14]:
# --- retrain cascades on full training data ---
def fit_and_score_cascade(model, X1_train, y1_train, X1_test, y1_test,
                          X2_train, y2_train, X2_test, y2_test):
    """
    Train a two-layer cascade and return final predicted probabilities for test set.
    """
    # Layer 1: BC->SC
    m1 = model.fit(X1_train, y1_train)
    p_sc = m1.predict_proba(X1_test)[:, 1]
    
    # Layer 2: SC->Contract (train only on SC=1 leads)
    mask_sc_train = (y1_train == 1)
    m2 = model.fit(X2_train[mask_sc_train], y2_train[mask_sc_train])
    p_sign_given_sc = m2.predict_proba(X2_test)[:, 1]
    
    # Final probability
    return p_sc * p_sign_given_sc

# Prepare train/test splits for both layers
drop_cols_layer1 = [
    'gross_fu','gross_sc','net_fu','net_sc',
    'time_first_sc_to_first_net_fu','selfipa_done'
]
X1_train = X_train.drop(columns=drop_cols_layer1)
X1_test  = X_test.drop(columns=drop_cols_layer1)
y1_train = (df.loc[X_train.index, 'net_sc'] >= 1).astype(int)
y1_test  = (df.loc[X_test.index, 'net_sc'] >= 1).astype(int)

X2_train, X2_test = X_train.copy(), X_test.copy()
y2_train, y2_test = y_train.copy(), y_test.copy()

# Models
baseline_model = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LogisticRegression(class_weight='balanced', max_iter=5000))
])

nn_model = Pipeline([
    ("scaler", StandardScaler()),
    ("nn", MLPClassifier(hidden_layer_sizes=(64,32),
                         activation='relu',
                         early_stopping=True,
                         random_state=42,
                         max_iter=200))
])

# Predictions
p_baseline = fit_and_score_cascade(baseline_model,
                                   X1_train, y1_train, X1_test, y1_test,
                                   X2_train, y2_train, X2_test, y2_test)

p_nn = fit_and_score_cascade(nn_model,
                             X1_train, y1_train, X1_test, y1_test,
                             X2_train, y2_train, X2_test, y2_test)

# Evaluate
for name, preds in [("Baseline", p_baseline), ("NN", p_nn)]:
    print(f"\n{name} Cascade on Test Set:")
    for pct in [0.01, 0.05, 0.10]:
        rec = recall_at_percent(y2_test, preds, top_percent=pct)
        lift = lift_at_percent(y2_test, preds, top_percent=pct)
        print(f"  Top {int(pct*100)}% -> Recall={rec:.3f}, Lift={lift:.2f}")



Baseline Cascade on Test Set:
  Top 1% -> Recall=0.008, Lift=0.87
  Top 5% -> Recall=0.017, Lift=0.34
  Top 10% -> Recall=0.068, Lift=0.68

NN Cascade on Test Set:
  Top 1% -> Recall=0.144, Lift=14.78
  Top 5% -> Recall=0.407, Lift=8.16
  Top 10% -> Recall=0.525, Lift=5.26


### Trying dynamic scoring (base + NN)

In [15]:
# 1) train models (baseline + NN for both layers, just like before).
# Train Layer 1 on BC-only features
drop_cols_layer1 = [
    'gross_fu','gross_sc','net_fu','net_sc',
    'time_first_sc_to_first_net_fu','selfipa_done'
]
X_bc = X.drop(columns=drop_cols_layer1)
y_sc = (df['net_sc'] >= 1).astype(int)

# Train Layer 2 on SC-leads
X_sc = X.copy()
y_sign = df['grosscontractsigned']

# Fit models (example: NN, but you can also do baseline)
model_layer1 = Pipeline([
    ("scaler", StandardScaler()),
    ("nn", MLPClassifier(hidden_layer_sizes=(64,32),
                         activation="relu",
                         early_stopping=True,
                         random_state=42,
                         max_iter=200))
]).fit(X_bc, y_sc)

model_layer2 = Pipeline([
    ("scaler", StandardScaler()),
    ("nn", MLPClassifier(hidden_layer_sizes=(64,32),
                         activation="relu",
                         early_stopping=True,
                         random_state=42,
                         max_iter=200))
]).fit(X_sc[y_sc==1], y_sign[y_sc==1])


In [16]:
# 2) scoring functions

def score_at_bc(model_layer1, X_bc, avg_sign_given_sc):
    """Score leads immediately after BC."""
    p_sc = model_layer1.predict_proba(X_bc)[:,1]
    # Rough proxy for sign likelihood
    p_sign_proxy = p_sc * avg_sign_given_sc
    return p_sc, p_sign_proxy

def score_at_sc(model_layer1, model_layer2, X_bc, X_sc):
    """Update scores for leads that reached SC."""
    p_sc = model_layer1.predict_proba(X_bc)[:,1]
    p_sign_given_sc = model_layer2.predict_proba(X_sc)[:,1]
    p_final = p_sc * p_sign_given_sc
    return p_final


In [17]:
# 3) using in practice

# Average conditional probability of signing given SC (from training set)
avg_sign_given_sc = y_sign[y_sc==1].mean()

# 1. At BC time (all leads)
p_sc, p_sign_proxy = score_at_bc(model_layer1, X_bc, avg_sign_given_sc)

# 2. Later, for those who reached SC
mask_sc = (df['net_sc'] >= 1)
p_final = score_at_sc(model_layer1, model_layer2, 
                      X_bc[mask_sc], X_sc[mask_sc])


In [18]:
# 4) evaluate separately 
# ---- 1. BC-time evaluation (all leads) ----
avg_sign_given_sc = y_sign[y_sc == 1].mean()
p_sc, p_sign_proxy = score_at_bc(model_layer1, X_bc, avg_sign_given_sc)

print("\nBC-time scoring (all leads):")
for pct in [0.01, 0.05, 0.10]:
    rec = recall_at_percent(y_sign, p_sign_proxy, top_percent=pct)
    lift = lift_at_percent(y_sign, p_sign_proxy, top_percent=pct)
    print(f"  Top {int(pct*100)}% -> Recall={rec:.3f}, Lift={lift:.2f}")

# ---- 2. SC-time evaluation (subset that reached SC) ----
mask_sc = (df['net_sc'] >= 1)
p_final = score_at_sc(model_layer1, model_layer2, X_bc[mask_sc], X_sc[mask_sc])

print("\nSC-time re-scoring (only leads that had >=1 SC):")
for pct in [0.01, 0.05, 0.10]:
    rec = recall_at_percent(y_sign[mask_sc], p_final, top_percent=pct)
    lift = lift_at_percent(y_sign[mask_sc], p_final, top_percent=pct)
    print(f"  Top {int(pct*100)}% -> Recall={rec:.3f}, Lift={lift:.2f}")



BC-time scoring (all leads):
  Top 1% -> Recall=0.025, Lift=2.55
  Top 5% -> Recall=0.075, Lift=1.49
  Top 10% -> Recall=0.164, Lift=1.64

SC-time re-scoring (only leads that had >=1 SC):
  Top 1% -> Recall=0.122, Lift=12.32
  Top 5% -> Recall=0.329, Lift=6.59
  Top 10% -> Recall=0.468, Lift=4.68


### NN Model tuning, Randomized Search for both layers

In [19]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

# -----------------
# Layer 1 (BC -> SC)
# -----------------
param_dist_layer1 = {
    "nn__hidden_layer_sizes": [(32,), (64,), (128,), (64,32), (128,64)],
    "nn__alpha": uniform(1e-5, 1e-2),          # L2 regularization
    "nn__learning_rate_init": uniform(1e-4, 1e-2),
    "nn__solver": ["adam", "lbfgs"]
}

nn_clf_layer1 = Pipeline([
    ("scaler", StandardScaler()),
    ("nn", MLPClassifier(
        activation="relu",
        early_stopping=True,
        max_iter=500, 
        random_state=42
    ))
])

search_layer1 = RandomizedSearchCV(
    nn_clf_layer1,
    param_distributions=param_dist_layer1,
    n_iter=20,            # try 20 random combos
    scoring="average_precision",  # PR-AUC as proxy metric
    n_jobs=-1,
    cv=5,
    random_state=42,
    verbose=2
)

search_layer1.fit(X_layer1, y_sc)

print("Best params for Layer 1:", search_layer1.best_params_)
print("Best score (PR-AUC) for Layer 1:", search_layer1.best_score_)

# -----------------
# Layer 2 (SC -> Contract)
# -----------------
param_dist_layer2 = {
    "nn__hidden_layer_sizes": [(32,), (64,), (128,), (64,32), (128,64)],
    "nn__alpha": uniform(1e-5, 1e-2),
    "nn__learning_rate_init": uniform(1e-4, 1e-2),
    "nn__solver": ["adam", "lbfgs"]
}

nn_clf_layer2 = Pipeline([
    ("scaler", StandardScaler()),
    ("nn", MLPClassifier(
        activation="relu",
        early_stopping=True,
        max_iter=500,
        random_state=42
    ))
])

mask_sc = (df['net_sc'] >= 1)   # only SC leads
search_layer2 = RandomizedSearchCV(
    nn_clf_layer2,
    param_distributions=param_dist_layer2,
    n_iter=20,
    scoring="average_precision",
    n_jobs=-1,
    cv=5,
    random_state=42,
    verbose=2
)

search_layer2.fit(X_layer2[mask_sc], y_sign[mask_sc])

print("Best params for Layer 2:", search_layer2.best_params_)
print("Best score (PR-AUC) for Layer 2:", search_layer2.best_score_)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best params for Layer 1: {'nn__alpha': np.float64(0.005934145688620425), 'nn__hidden_layer_sizes': (128,), 'nn__learning_rate_init': np.float64(0.008699404067363204), 'nn__solver': 'adam'}
Best score (PR-AUC) for Layer 1: 0.7071305107528186
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best params for Layer 2: {'nn__alpha': np.float64(0.00231893825622149), 'nn__hidden_layer_sizes': (64, 32), 'nn__learning_rate_init': np.float64(0.004501524937396013), 'nn__solver': 'adam'}
Best score (PR-AUC) for Layer 2: 0.22126000855818034


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best params for Layer 1: {'nn__alpha': np.float64(0.005934145688620425), 'nn__hidden_layer_sizes': (128,), 'nn__learning_rate_init': np.float64(0.008699404067363204), 'nn__solver': 'adam'}
Best score (PR-AUC) for Layer 1: 0.7071305107528186
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best params for Layer 2: {'nn__alpha': np.float64(0.00231893825622149), 'nn__hidden_layer_sizes': (64, 32), 'nn__learning_rate_init': np.float64(0.004501524937396013), 'nn__solver': 'adam'}
Best score (PR-AUC) for Layer 2: 0.22126000855818034

In [20]:
# with these params
# --- retrain with best hyperparams ---

# Layer 1: BC → SC
best_layer1 = Pipeline([
    ("scaler", StandardScaler()),
    ("nn", MLPClassifier(
        hidden_layer_sizes=(128,),
        alpha=0.005934145688620425,
        learning_rate_init=0.008699404067363204,
        solver="adam",
        activation="relu",
        early_stopping=True,
        max_iter=500,
        random_state=42
    ))
])
best_layer1.fit(X_layer1, y_sc)

# Layer 2: SC → Contract
mask_sc = (df['net_sc'] >= 1)
best_layer2 = Pipeline([
    ("scaler", StandardScaler()),
    ("nn", MLPClassifier(
        hidden_layer_sizes=(64, 32),
        alpha=0.00231893825622149,
        learning_rate_init=0.004501524937396013,
        solver="adam",
        activation="relu",
        early_stopping=True,
        max_iter=500,
        random_state=42
    ))
])
best_layer2.fit(X_layer2[mask_sc], y_sign[mask_sc])

# --- evaluation ---
print("\nBC-time scoring (all leads, tuned Layer 1):")
avg_sign_given_sc = y_sign[mask_sc].mean()
p_sc, p_sign_proxy = score_at_bc(best_layer1, X_layer1, avg_sign_given_sc)

for pct in [0.01, 0.05, 0.10]:
    rec = recall_at_percent(y_sign, p_sign_proxy, top_percent=pct)
    lift = lift_at_percent(y_sign, p_sign_proxy, top_percent=pct)
    print(f"  Top {int(pct*100)}% -> Recall={rec:.3f}, Lift={lift:.2f}")

print("\nSC-time re-scoring (only SC leads, tuned Layer 1 + Layer 2):")
p_final = score_at_sc(best_layer1, best_layer2, X_layer1[mask_sc], X_layer2[mask_sc])

for pct in [0.01, 0.05, 0.10]:
    rec = recall_at_percent(y_sign[mask_sc], p_final, top_percent=pct)
    lift = lift_at_percent(y_sign[mask_sc], p_final, top_percent=pct)
    print(f"  Top {int(pct*100)}% -> Recall={rec:.3f}, Lift={lift:.2f}")



BC-time scoring (all leads, tuned Layer 1):
  Top 1% -> Recall=0.022, Lift=2.21
  Top 5% -> Recall=0.081, Lift=1.63
  Top 10% -> Recall=0.142, Lift=1.42

SC-time re-scoring (only SC leads, tuned Layer 1 + Layer 2):
  Top 1% -> Recall=0.120, Lift=12.15
  Top 5% -> Recall=0.342, Lift=6.86
  Top 10% -> Recall=0.497, Lift=4.97


hyperparam tuning wasnt a big gain

### NN imbalance handling. New model for Focal loss

In [22]:
#Define focal loss

import tensorflow as tf
from tensorflow.keras import backend as K

def focal_loss(gamma=2., alpha=.25):
    def loss(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        ce = -y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred)
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        fl = ce * ((1 - p_t) ** gamma)
        if alpha is not None:
            alpha_t = y_true * alpha + (1 - y_true) * (1 - alpha)
            fl *= alpha_t
        return K.mean(fl)
    return loss


In [23]:
# Build a simple NN model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

def make_model(input_dim, hidden_layers=(64,32), alpha=0.25, gamma=2.0):
    model = Sequential()
    for i, units in enumerate(hidden_layers):
        model.add(Dense(units, activation="relu", input_dim=input_dim if i==0 else None))
        model.add(Dropout(0.2))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(
        optimizer="adam",
        loss=focal_loss(gamma=gamma, alpha=alpha),
        metrics=[tf.keras.metrics.AUC(name="auc"), tf.keras.metrics.Precision(name="precision"), tf.keras.metrics.Recall(name="recall")]
    )
    return model


Training flow

Layer 1: train on X_layer1, y_sc

Layer 2: train on X_layer2[mask_sc], y_sign[mask_sc]

In [24]:
# Layer 1 model
model_layer1 = make_model(input_dim=X_layer1.shape[1], hidden_layers=(128,))
model_layer1.fit(X_layer1, y_sc, 
                 validation_split=0.2,
                 epochs=50, batch_size=128, verbose=2,
                 callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)])

# Layer 2 model
mask_sc = (df['net_sc'] >= 1)
model_layer2 = make_model(input_dim=X_layer2.shape[1], hidden_layers=(64,32))
model_layer2.fit(X_layer2[mask_sc], y_sign[mask_sc],
                 validation_split=0.2,
                 epochs=50, batch_size=128, verbose=2,
                 callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)])


Epoch 1/50


c:\Users\ElifYilmaz\OneDrive - Enpal B.V\Desktop\New folder\Project\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


113/113 - 1s - 10ms/step - auc: 0.5199 - loss: 0.1201 - precision: 0.6204 - recall: 0.2824 - val_auc: 0.5925 - val_loss: 0.0726 - val_precision: 0.8000 - val_recall: 0.0773
Epoch 2/50
113/113 - 0s - 2ms/step - auc: 0.5435 - loss: 0.1007 - precision: 0.6506 - recall: 0.2271 - val_auc: 0.6094 - val_loss: 0.0711 - val_precision: 0.8767 - val_recall: 0.0302
Epoch 3/50
113/113 - 0s - 2ms/step - auc: 0.5689 - loss: 0.0959 - precision: 0.6852 - recall: 0.1792 - val_auc: 0.6050 - val_loss: 0.0712 - val_precision: 0.7756 - val_recall: 0.1107
Epoch 4/50
113/113 - 0s - 3ms/step - auc: 0.5708 - loss: 0.0961 - precision: 0.7003 - recall: 0.1278 - val_auc: 0.6197 - val_loss: 0.0702 - val_precision: 0.8380 - val_recall: 0.0561
Epoch 5/50
113/113 - 0s - 3ms/step - auc: 0.5797 - loss: 0.0908 - precision: 0.7098 - recall: 0.0953 - val_auc: 0.6171 - val_loss: 0.0701 - val_precision: 0.9057 - val_recall: 0.0452
Epoch 6/50
113/113 - 0s - 3ms/step - auc: 0.5843 - loss: 0.0903 - precision: 0.7185 - recall: 0

c:\Users\ElifYilmaz\OneDrive - Enpal B.V\Desktop\New folder\Project\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


67/67 - 2s - 26ms/step - auc: 0.5368 - loss: 0.0481 - precision: 0.0457 - recall: 0.0162 - val_auc: 0.5656 - val_loss: 0.0190 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/50
67/67 - 0s - 4ms/step - auc: 0.5748 - loss: 0.0318 - precision: 0.0556 - recall: 0.0020 - val_auc: 0.6304 - val_loss: 0.0186 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/50
67/67 - 0s - 4ms/step - auc: 0.6002 - loss: 0.0281 - precision: 0.2500 - recall: 0.0020 - val_auc: 0.7156 - val_loss: 0.0180 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/50
67/67 - 0s - 4ms/step - auc: 0.6188 - loss: 0.0264 - precision: 0.1429 - recall: 0.0020 - val_auc: 0.7316 - val_loss: 0.0180 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 5/50
67/67 - 0s - 4ms/step - auc: 0.6466 - loss: 0.0250 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_auc: 0.7502 - val_loss: 0.0168 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 6/50
67/67 - 0s - 4ms/step - auc: 0.6509 - loss: 0.

In [25]:
#Scoring functions
def score_at_bc_keras(model_layer1, X_bc, avg_sign_given_sc):
    p_sc = model_layer1.predict(X_bc).flatten()
    p_sign_proxy = p_sc * avg_sign_given_sc
    return p_sc, p_sign_proxy

def score_at_sc_keras(model_layer1, model_layer2, X_bc, X_sc):
    p_sc = model_layer1.predict(X_bc).flatten()
    p_sign_given_sc = model_layer2.predict(X_sc).flatten()
    p_final = p_sc * p_sign_given_sc
    return p_final


In [26]:
# Evaluate BC vs SC performance
# Average conditional sign rate (for BC proxy score)
avg_sign_given_sc = y_sign[mask_sc].mean()

# 1. BC-time evaluation
p_sc, p_sign_proxy = score_at_bc_keras(model_layer1, X_layer1, avg_sign_given_sc)
print("\nBC-time scoring (all leads, focal loss):")
for pct in [0.01, 0.05, 0.10]:
    rec = recall_at_percent(y_sign, p_sign_proxy, top_percent=pct)
    lift = lift_at_percent(y_sign, p_sign_proxy, top_percent=pct)
    print(f"  Top {int(pct*100)}% -> Recall={rec:.3f}, Lift={lift:.2f}")

# 2. SC-time evaluation
p_final = score_at_sc_keras(model_layer1, model_layer2, X_layer1[mask_sc], X_layer2[mask_sc])
print("\nSC-time re-scoring (only SC leads, focal loss):")
for pct in [0.01, 0.05, 0.10]:
    rec = recall_at_percent(y_sign[mask_sc], p_final, top_percent=pct)
    lift = lift_at_percent(y_sign[mask_sc], p_final, top_percent=pct)
    print(f"  Top {int(pct*100)}% -> Recall={rec:.3f}, Lift={lift:.2f}")


562/562 ━━━━━━━━━━━━━━━━━━━━ 0s 746us/step

BC-time scoring (all leads, focal loss):
  Top 1% -> Recall=0.012, Lift=1.19
  Top 5% -> Recall=0.080, Lift=1.59
  Top 10% -> Recall=0.154, Lift=1.54
335/335 ━━━━━━━━━━━━━━━━━━━━ 0s 548us/step
335/335 ━━━━━━━━━━━━━━━━━━━━ 0s 596us/step

SC-time re-scoring (only SC leads, focal loss):
  Top 1% -> Recall=0.054, Lift=5.47
  Top 5% -> Recall=0.249, Lift=4.99
  Top 10% -> Recall=0.375, Lift=3.75


🔎 Interpretation

focal loss is very useful when models collapse to predicting negatives everywhere.

but your NN (even untuned) was already separating signal well enough, so focal loss may have been too aggressive and actually hurt ranking performance.

with ~3% conversion, you’re in a “rare but not ultra-rare” regime — class weights or sample weights often do just as well, sometimes better.

### Step back from Keras (focal loss) because no improvement. Class weights for first NN model

In [27]:
# compute weights
from sklearn.utils.class_weight import compute_sample_weight

# for layer 1 (BC→SC)
sw_layer1 = compute_sample_weight(class_weight="balanced", y=y_sc)

# for layer 2 (SC→Contract)
mask_sc = (df['net_sc'] >= 1)
sw_layer2 = compute_sample_weight(class_weight="balanced", y=y_sign[mask_sc])

#retrain NN with weights

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

# layer 1 model
nn_layer1 = Pipeline([
    ("scaler", StandardScaler()),
    ("nn", MLPClassifier(hidden_layer_sizes=(128,),
                         activation="relu",
                         early_stopping=True,
                         max_iter=500,
                         random_state=42))
])
nn_layer1.fit(X_layer1, y_sc, nn__sample_weight=sw_layer1)

# layer 2 model
nn_layer2 = Pipeline([
    ("scaler", StandardScaler()),
    ("nn", MLPClassifier(hidden_layer_sizes=(64,32),
                         activation="relu",
                         early_stopping=True,
                         max_iter=500,
                         random_state=42))
])
nn_layer2.fit(X_layer2[mask_sc], y_sign[mask_sc], nn__sample_weight=sw_layer2)

#evaluate again

# 1. BC-time
avg_sign_given_sc = y_sign[mask_sc].mean()
p_sc = nn_layer1.predict_proba(X_layer1)[:,1]
p_sign_proxy = p_sc * avg_sign_given_sc

print("\nBC-time scoring (all leads, sklearn NN + weights):")
for pct in [0.01, 0.05, 0.10]:
    rec = recall_at_percent(y_sign, p_sign_proxy, top_percent=pct)
    lift = lift_at_percent(y_sign, p_sign_proxy, top_percent=pct)
    print(f"  Top {int(pct*100)}% -> Recall={rec:.3f}, Lift={lift:.2f}")

# 2. SC-time
p_sc = nn_layer1.predict_proba(X_layer1[mask_sc])[:,1]
p_sign_given_sc = nn_layer2.predict_proba(X_layer2[mask_sc])[:,1]
p_final = p_sc * p_sign_given_sc

print("\nSC-time re-scoring (only SC leads, sklearn NN + weights):")
for pct in [0.01, 0.05, 0.10]:
    rec = recall_at_percent(y_sign[mask_sc], p_final, top_percent=pct)
    lift = lift_at_percent(y_sign[mask_sc], p_final, top_percent=pct)
    print(f"  Top {int(pct*100)}% -> Recall={rec:.3f}, Lift={lift:.2f}")




BC-time scoring (all leads, sklearn NN + weights):
  Top 1% -> Recall=0.020, Lift=2.04
  Top 5% -> Recall=0.078, Lift=1.56
  Top 10% -> Recall=0.144, Lift=1.44

SC-time re-scoring (only SC leads, sklearn NN + weights):
  Top 1% -> Recall=0.037, Lift=3.76
  Top 5% -> Recall=0.210, Lift=4.21
  Top 10% -> Recall=0.351, Lift=3.51


At BC-time: class weights don’t hurt or help much (features too weak).

At SC-time: plain tuned NN is still your best performer so far.

Focal loss → too aggressive.

Class weights → overcorrected, lost ranking sharpness.

Plain tuned NN (no imbalance tricks) gave the strongest lift.

## XGBoost Cascade Pipeline (Comparing NN to a tree based model, 2nd benchmark)

In [28]:
import xgboost as xgb
from sklearn.metrics import average_precision_score

# ------------------
# Layer 1: BC → SC
# ------------------
scale_pos_weight_sc = (len(y_sc) - y_sc.sum()) / y_sc.sum()

model_sc = xgb.XGBClassifier(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    scale_pos_weight=scale_pos_weight_sc,
    random_state=42,
    use_label_encoder=False,
    eval_metric="logloss"
)

model_sc.fit(X_layer1.loc[X_train.index], (df.loc[X_train.index, 'net_sc'] >= 1).astype(int))

# ------------------
# Layer 2: SC → Contract
# ------------------
mask_train_sc = (df.loc[X_train.index, 'net_sc'] >= 1)
scale_pos_weight_sign = (mask_train_sc.sum() - y_train[mask_train_sc].sum()) / y_train[mask_train_sc].sum()

model_sign = xgb.XGBClassifier(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    scale_pos_weight=scale_pos_weight_sign,
    random_state=42,
    use_label_encoder=False,
    eval_metric="logloss"
)

model_sign.fit(X_layer2.loc[X_train.index][mask_train_sc], y_train[mask_train_sc])

# ------------------
# Evaluation on test set
# ------------------
# Layer 1 probs
p_sc = model_sc.predict_proba(X_layer1.loc[X_test.index])[:, 1]

# Layer 2 probs (only for test leads that actually had SC)
mask_test_sc = (df.loc[X_test.index, 'net_sc'] >= 1)
p_sign_cond_sc = np.zeros(len(X_test))
p_sign_cond_sc[mask_test_sc] = model_sign.predict_proba(X_layer2.loc[X_test.index][mask_test_sc])[:, 1]

# Final cascade probability = P(SC) * P(Sign|SC)
p_final = p_sc * p_sign_cond_sc

# ------------------
# Metrics
# ------------------
for top in [0.01, 0.05, 0.10]:
    rec = recall_at_percent(y_test, p_final, top)
    lift = lift_at_percent(y_test, p_final, top)
    print(f"Top {int(top*100)}% → Recall={rec:.3f}, Lift={lift:.2f}")


c:\Users\ElifYilmaz\OneDrive - Enpal B.V\Desktop\New folder\Project\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [10:29:25] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\ElifYilmaz\OneDrive - Enpal B.V\Desktop\New folder\Project\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [10:29:25] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Top 1% → Recall=0.051, Lift=5.22
Top 5% → Recall=0.280, Lift=5.61
Top 10% → Recall=0.466, Lift=4.66


In [29]:
# ------------------
# BC-time scoring (all leads, proxy contract score)
# ------------------
# At BC we only know P(SC). We don't know P(Sign|SC) yet,
# so a simple proxy is: P(contract) ≈ P(SC) * avg(sign rate given SC in train)
avg_sign_given_sc = y_train[df.loc[X_train.index, 'net_sc'] >= 1].mean()

p_bc_proxy = p_sc * avg_sign_given_sc

print("\nBC-time scoring (all leads):")
for top in [0.01, 0.05, 0.10]:
    rec = recall_at_percent(y_test, p_bc_proxy, top)
    lift = lift_at_percent(y_test, p_bc_proxy, top)
    print(f"  Top {int(top*100)}% -> Recall={rec:.3f}, Lift={lift:.2f}")

# ------------------
# SC-time re-scoring (only leads that had >=1 SC)
# ------------------
mask_test_sc = (df.loc[X_test.index, 'net_sc'] >= 1)
p_sc_only = model_sign.predict_proba(X_layer2.loc[X_test.index][mask_test_sc])[:, 1]

print("\nSC-time re-scoring (only SC leads):")
for top in [0.01, 0.05, 0.10]:
    rec = recall_at_percent(y_test[mask_test_sc], p_sc_only, top)
    lift = lift_at_percent(y_test[mask_test_sc], p_sc_only, top)
    print(f"  Top {int(top*100)}% -> Recall={rec:.3f}, Lift={lift:.2f}")



BC-time scoring (all leads):
  Top 1% -> Recall=0.008, Lift=0.87
  Top 5% -> Recall=0.051, Lift=1.02
  Top 10% -> Recall=0.119, Lift=1.19

SC-time re-scoring (only SC leads):
  Top 1% -> Recall=0.051, Lift=5.25
  Top 5% -> Recall=0.314, Lift=6.29
  Top 10% -> Recall=0.415, Lift=4.17
